In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import math
from torch.distributions import constraints

import pyro
import pyro.distributions as pydist
from pyro.infer import SVI, Trace_ELBO


# define a model to generate data: A simple sinusoid

def sine_model(t, t0, omega, A, phi0):
    
    return A * torch.sin(omega * (t - t0) - phi0)

def add_noise(x, var):
    
    return x + torch.normal(0.0, math.sqrt(var), size=x.shape)


A = 1
omega = 2 * torch.pi * 3e6
f_sample = 10e6
n_sample = 8192
t = torch.arange(0, n_sample, 1) / f_sample
t0 = 0
phi0 = 0

var = 1e-1

data = sine_model(t, t0, omega, A, phi0)
data = add_noise(data, var)

# define a probalistic model for the data
def model(data):
    
    # priors
    A = pyro.param("A_latent", torch.tensor(0.95), constraints.interval(0.9, 1.1))
    omega = pyro.param("omega_latent", torch.tensor(2*torch.pi*2.88e6), constraints.interval(2*torch.pi*2.85e6, 2*torch.pi*3.15e6))
    #t0 = pyro.sample("t0_latent", pydist.Uniform(0, 512))
    phi0 = pyro.param("phi0_latent",torch.tensor(0, constraints.interval(-torch.pi, torch.pi)))
    # known parameters
    var = 1e-1
    n_sample = 16384
    f_sample = 10e6
    t = torch.arange(0, n_sample, 1) / f_sample
    
    mean = data - A * torch.sin(omega * t - phi0)
    
    pyro.sample('obs', pydist.MultivariateNormal(mean, covariance_matrix = torch.eye(mean.shape[-1]) * var), obs=data)

def train(model, guide, lr=0.005, n_steps=301):
    pyro.clear_param_store()
    adam_params = {"lr": lr}
    adam = pyro.optim.Adam(adam_params)
    svi = SVI(model, guide, adam, loss=Trace_ELBO())

    for step in range(n_steps):
        loss = svi.step(data)
        if step % 50 == 0:
            print('[iter {}]  loss: {:.4f}'.format(step, loss))
    
# use autoguide
#autoguide_map = pyro.infer.autoguide.AutoDelta(model)

def guide_mle(data):
    pass

train(model, guide_mle)

sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)

In [ ]:
print(autoguide_map.median(data))

In [ ]:
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(t, data)
ax.plot(t, sine_model(t, 0,2065578.1250, 0.9, 0.1962 ))

In [ ]:
2*torch.pi*0.25e6


In [ ]:
A = 1
omega = 2 * torch.pi * 0.25e6
f_sample = 10e6
n_sample = 1024
t = torch.arange(0, n_sample, 1) / f_sample
t0 = 256/f_sample
phi0 = 0

var = 1e-1

x = sine_model(t, t0, omega, A, phi0)
x = add_noise(x, var)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(t, x)